In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np

2025-01-18 09:37:13.734252: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Set up directories
train_dir = r"/Users/pramodjumde/Downloads/archive (8)/train"
valid_dir = r"/Users/pramodjumde/Downloads/archive (8)/test"
test_dir = r"/Users/pramodjumde/Downloads/archive (8)/valid"

# Set up ImageDataGenerators for Loading images
train_datagen = ImageDataGenerator(rescale = 1./255)
valid_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(train_dir, target_size = (64, 64), class_mode = 'binary')
valid_generator = valid_datagen.flow_from_directory(train_dir, target_size = (64, 64), class_mode = 'binary')
test_generator = test_datagen.flow_from_directory(train_dir, target_size = (64, 64), class_mode = 'binary')

Found 30250 images belonging to 2 classes.
Found 30250 images belonging to 2 classes.
Found 30250 images belonging to 2 classes.


In [5]:
# Building a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation = 'relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size = (2, 2)),
    Conv2D(64, (3, 3), activation = 'relu'),
    MaxPooling2D(pool_size = (2, 2)),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dropout(0.5),
    Dense(1, activation = 'sigmoid')    # Binary Classification: Wildfire or No Wildfire
])

# Compile the model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

/Users/pramodjumde/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [9]:
# Train the model
history = model.fit(train_generator, validation_data = valid_generator, epochs = 10, verbose = 1)

/Users/pramodjumde/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 433s 455ms/step - accuracy: 0.8706 - loss: 0.2972 - val_accuracy: 0.9361 - val_loss: 0.1715
Epoch 2/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 416s 440ms/step - accuracy: 0.9312 - loss: 0.1806 - val_accuracy: 0.9333 - val_loss: 0.1837
Epoch 3/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 439s 465ms/step - accuracy: 0.9383 - loss: 0.1673 - val_accuracy: 0.9485 - val_loss: 0.1351
Epoch 4/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 407s 430ms/step - accuracy: 0.9480 - loss: 0.1438 - val_accuracy: 0.9233 - val_loss: 0.1959
Epoch 5/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 410s 434ms/step - accuracy: 0.9487 - loss: 0.1414 - val_accuracy: 0.9612 - val_loss: 0.1129
Epoch 6/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 407s 431ms/step - accuracy: 0.9545 - loss: 0.1224 - val_accuracy: 0.9646 - val_loss: 0.0975
Epoch 7/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 424s 448ms/step - accuracy: 0.9574 - loss: 0.1150 - val_accuracy: 0.9706 - val_loss: 0.0816
Epoch 8/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 438s 463ms/step - accuracy: 0.9630 -

In [12]:
def predict_image():
    file_path=filedialog.askopenfilename()
    if file_path:
        img=Image.open(file_path)
        img=img.resize((200,200))
        img=ImageTk.PhotoImage(img)
        image_label.configure(image=img)
        image_label.image=img
        img_for_model=Image.open(file_path).resize((64,64))
        img_array=np.array(img_for_model)/255.0
        img_array=np.expand_dims(img_array,axis=0)
        prediction=model.predict(img_array)[0][0]
        result="Wildfire" if prediction>0.5 else "No Wildfire"
        result_label.config(text="Prediction: "+result)
root=tk.Tk()
root.title("Forest Fire Detection")
root.geometry("400x400")

btn=tk.Button(root,text="Upload Image",command=predict_image)
btn.pack(pady=20)

image_label=tk.Label(root)
image_label.pack()

result_label=tk.Label(root,text="Prediction: ",font=("Helvetica",16))
result_label.pack(pady=20)

root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step
